In [ ]:
import os
os.environ["DJANGO_SETTINGS_MODULE"] = "ClusterCast.settings"
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from django.db.models.functions import Now
from asgiref.sync import sync_to_async
from tensorflow.keras.layers import Input, LSTM, Dropout, RepeatVector, TimeDistributed, Dense, Masking
from tensorflow.keras.models import Model
import ClusterPipeline.models.RNNModels as rnn
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.RNNModels as rnn 

In [ ]:
@sync_to_async
def get_all_objects(features,steps = None):
    # Force the query to execute and load all results into memory
    params = list(cp.StockClusterGroupParams.objects.all())
    matching_groups = []
    for param in params:
        if steps is None:
            if param.cluster_features == features:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()

        else: 
            if param.cluster_features == features and param.n_steps == steps:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()
                print(group.group_params.tickers)
    
    return matching_groups


cluster_features = ["close", "bb_low", "bb_high"]
steps = 20

async def create_cluster_group_params(cluster_features, steps=20):

    cluster_groups = await get_all_objects(features=cluster_features,steps=steps)
    return cluster_groups

# Run the async function

matching_groups = await create_cluster_group_params(cluster_features, steps=steps)

In [ ]:
@sync_to_async
def get_all_clusters(groups):
    clusters = []
    features = []
    for group in groups:
        clusters += group.clusters

    for cluster in clusters:
        models = rnn.RNNModel.objects.filter(cluster=cluster)
        for model in models: 
            if model:
                features += (model.model_features)

    features = list(set(features))
    return clusters, features

clusters, all_features = await get_all_clusters(matching_groups)



In [ ]:
import numpy as np

# Function to pad a 2D array to have the target number of rows
def pad_array(lst, target_rows):
    # Calculate how many rows to pad
    arr = np.array(lst)
    padding_rows = target_rows - arr.shape[0]
    # Apply padding only to the rows
    return np.pad(arr, pad_width=((0, padding_rows), (0, 0)), mode='constant', constant_values=0)

max_rows = max(len(cluster.centroid) for cluster in clusters)
padded_centroids = [pad_array(cluster.centroid, max_rows) for cluster in clusters]

# Convert the list of 2D arrays to a 3D numpy array
centroids_3d = np.stack(padded_centroids)
centroids_3d.shape



In [ ]:
from tslearn.clustering import TimeSeriesKMeans
def cluster_centroids(centroids_3d, clusters,n_clusters=10):
    # Create the clustering model
    cluster_alg = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=5, random_state=0)
    # Fit the clustering model
    labels = cluster_alg.fit_predict(centroids_3d)
    # Get the cluster centroids
    centroids = cluster_alg.cluster_centers_

    centroid_group = {}
    for cluster,label in zip(clusters,labels):
        if label not in centroid_group:
            centroid_group[label] = []
        centroid_group[label].append(cluster)
    

    return centroid_group, centroids


In [ ]:
first_centroid_group,centroids = cluster_centroids(centroids_3d, clusters, n_clusters=8)

In [ ]:
print(first_centroid_group)

In [ ]:
first_label = first_centroid_group[0]

In [ ]:
# get n random features from features list 
import random
from scipy.interpolate import interp1d
from copy import deepcopy

def front_pad_3d_array(arr, target_cols):
    padding_cols = target_cols - arr.shape[1]
    pad_width = ((0, 0), (padding_cols, 0), (0, 0))  # Front padding
    return np.pad(arr, pad_width=pad_width, mode='constant', constant_values=0.0)
def back_pad_3d_array(arr, target_cols):
    padding_cols = target_cols - arr.shape[1]
    pad_width = ((0, 0), (0, padding_cols), (0, 0))  # Back padding
    return np.pad(arr, pad_width=pad_width, mode='constant', constant_values=0.0)

def resample_time_series(data, max_time_steps):
    data = deepcopy(data)
    n_elements, current_time_steps, n_features = data.shape
    resampled_data = np.zeros((n_elements, max_time_steps, n_features))

    for i in range(n_elements):
        for j in range(n_features):
            x = np.linspace(0, 1, current_time_steps)
            x_new = np.linspace(0, 1, max_time_steps)
            f = interp1d(x, data[i, :, j], kind='linear')
            resampled_data[i, :, j] = f(x_new)

    return resampled_data

max_cols = max(cluster.X_train.shape[1] for cluster in first_label)
# Pad each X_train and collect them in a list
for cluster in first_label:
    cluster.X_train_resampled = resample_time_series(cluster.X_train, max_cols)
    cluster.X_test_resampled = resample_time_series(cluster.X_test, max_cols)


# Concatenate the list of padded X_train arrays 

In [ ]:
X_train_filtered = []
y_train_filtered = []
X_test_filtered = []
y_test_filtered = []
features = random.sample(all_features, 75)
for cluster in first_label:
    feautre_dict = cluster.X_feature_dict
    y_train = cluster.y_train
    y_test = cluster.y_test
    print(feautre_dict)
    cols_indices = [feautre_dict[feature] for feature in features]

    filtered_train_sampled = cluster.X_train_resampled[:,:,cols_indices]
    filtered_test_sampled = cluster.X_test_resampled[:,:,cols_indices]
    
    cluster.X_train_filtered_sampled = filtered_train_sampled
    cluster.X_test_filtered_sampled = filtered_test_sampled

    cluster.X_train_filtered = cluster.X_train[:,:,cols_indices]
    cluster.X_test_filtered = cluster.X_test[:,:,cols_indices]
    # print(str(filtered.shape) + " " + str(X_train.shape))
    X_train_filtered.append(filtered_train_sampled)
    y_train_filtered.append(y_train)
    X_test_filtered.append(filtered_test_sampled)
    y_test_filtered.append(y_test)

all_X_train = np.concatenate(X_train_filtered)
all_X_train.shape
all_y_train = np.concatenate(y_train_filtered)

all_X_test = np.concatenate(X_test_filtered)
all_y_test = np.concatenate(y_test_filtered)

In [ ]:
import matplotlib.pyplot as plt
first_cluster = first_label[1]
plt.plot(first_cluster.X_train_filtered_sampled[0,:,-1])
plt.show()
plt.plot(first_cluster.X_train_filtered[0,:,-1])
plt.show()
# graph the

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Masking
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.layers import Layer



In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, TimeDistributed, Dense, Concatenate, Permute, Reshape, Multiply
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.layers import Layer
from keras.layers import Activation, Flatten
def create_model(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # masking_layer = Masking(mask_value=0.0, name='masking_layer')(input_layer)

    # Encoder
    encoder_lstm1 = LSTM(units=500, activation='tanh', return_sequences=True, name='encoder_lstm_1_freeze')(input_layer)
    encoder_dropout1 = Dropout(0.2, name='encoder_dropout_1_freeze')(encoder_lstm1)

    encoder_lstm2 = LSTM(units=100, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(encoder_dropout1)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=100, activation='tanh', return_sequences=True, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=False, name='encoder_lstm_4_restore')(encoder_dropout3)
    encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Repeat Vector
    repeat_vector = RepeatVector(latent_dim, name='repeat_vector')(encoder_dropout4)

    # Decoder
    decoder_lstm1 = LSTM(units=6, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(repeat_vector)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    # decoder_lstm2 = LSTM(units=50, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    # decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    # decoder_lstm3 = LSTM(units=6, activation='tanh', return_sequences=True, name='decoder_lstm_3_restore')(decoder_dropout2)
    # decoder_dropout2 = Dropout(0.2, name='decoder_dropout_3_restore')(decoder_lstm3)

    

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout1)

    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=time_distributed_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mse")

    return model_lstm


In [ ]:

# Custom Attention Layer

def create_model_with_attention2(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # Encoder

    encoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_1_restore')(input_layer)
    encoder_dropout1 = Dropout(0.2, name='encoder_dropout_1_restore')(encoder_lstm1)

    encoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(encoder_dropout1)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=10, activation='tanh', return_sequences=True, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    output_lstm = LSTM(units=6, activation='tanh', return_sequences=True, name='outputLSTM')(encoder_dropout3)
    encoder_output = Dropout(0.2, name='encoder_output')(output_lstm)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Attention Layer
    # attention = AttentionLayer(name='attention_layer')(encoder_dropout4)
     # Attention Mechanism
    attention = Dense(1, activation='tanh')(encoder_output)
    attention = Flatten()(attention)
    attention_weights = Activation('softmax')(attention)
    context = Multiply()([encoder_output, Permute([2, 1])(RepeatVector(6)(attention_weights))])

    # Decoder
    decoder_lstm1 = LSTM(units=75, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(context)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    decoder_lstm3 = LSTM(units=10, activation='tanh', return_sequences=True, name='decoder_lstm_3_restore')(decoder_dropout2)
    decoder_dropout3 = Dropout(0.2, name='decoder_dropout_3_restore')(decoder_lstm3)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout3)

    final_output = time_distributed_output[:, -6:, :]

    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=final_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mse")

    return model_lstm


In [ ]:
import copy
def save_decoder_initial_weights(model):
    initial_weights = {}
    for layer in model.layers:
        if 'restore' in layer.name:  # Assuming decoder layers have 'decoder' in their names
            initial_weights[layer.name] = copy.deepcopy(layer.get_weights())
    return initial_weights

model = create_model_with_attention2(len(features))
decoder_weights = save_decoder_initial_weights(model)

In [29]:
print(all_X_train.shape)
print(all_y_train.shape)

(1086, 20, 75)
(1086, 6)


In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
model.fit(all_X_train, all_y_train,validation_data=(all_X_test,all_y_test), epochs=40, batch_size=32, verbose=1)
for layer in model.layers:
    if 'freeze' in layer.name:
        print(layer.name)
        layer.trainable = False

In [ ]:
model.save('path_to_my_model.h5')

In [ ]:
import pandas as pd 
def eval_model(X_test, y_test, model,cluster):

    predicted_y = model.predict(X_test)
    predicted_y = np.squeeze(predicted_y, axis=-1)

    num_days = predicted_y.shape[1]  # Assuming this is the number of days
    results = pd.DataFrame(predicted_y, columns=[f'predicted_{i+1}' for i in range(num_days)])

    for i in range(num_days):
        results[f'real_{i+1}'] = y_test[:, i]

    # Generate output string with accuracies
    output_string = f"Cluster Number: {cluster.label}\n"
    for i in range(num_days):
        same_day = ((results[f'predicted_{i+1}'] > 0) & (results[f'real_{i+1}'] > 0)) | \
                ((results[f'predicted_{i+1}'] < 0) & (results[f'real_{i+1}'] < 0))
        accuracy = round(same_day.mean() * 100,2)

        output_string += (
            f"Accuracy{i+1}D {accuracy}% "
            f"PredictedRet: {results[f'predicted_{i+1}'].mean()} "
            f"ActRet: {results[f'real_{i+1}'].mean()}\n"
        )
    
    output_string += f"Train set length: {len(X_train_filtered)} Test set length: {len(y_test)}\n"

    return output_string, results

In [ ]:
first_cluster = first_label[2]
X_train_cluster = first_cluster.X_train_filtered_sampled
y_train_cluster = first_cluster.y_train
X_test_cluster = first_cluster.X_test_filtered_sampled
y_test_cluster = first_cluster.y_test
print(X_train_cluster.shape)
print(y_train_cluster.shape)
print(X_test_cluster.shape)
print(y_test_cluster.shape)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('path_to_my_model.h5')
for layer in model.layers:
    if 'restore' in layer.name:
        layer.set_weights(decoder_weights[layer.name])


In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse')

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

# Fine-tune the model using your smaller dataset
# Assume you have 'small_train_data', 'small_train_labels', 'small_val_data', and 'small_val_labels'
history = model.fit(X_train_cluster, y_train_cluster,
                    validation_data=(X_test_cluster, y_test_cluster),
                    epochs=250,
                    batch_size=16,
                    callbacks=[early_stopping])

# Evaluate the model performance
val_loss = model.evaluate(X_test_cluster, y_test_cluster)
print(f'Validation loss: {val_loss}')
fineTunedAccuracy, results_tuned = eval_model(X_test_cluster, y_test_cluster, model,first_cluster)
del model
clear_session()

In [ ]:
benchMarkModel = create_model(len(features))

In [ ]:
optimizer = Adam(learning_rate=0.0001)
benchMarkModel.compile(optimizer=optimizer, loss='mse')

X_train = first_cluster.X_train_filtered
y_train = first_cluster.y_train
X_test = first_cluster.X_test_filtered
y_test = first_cluster.y_test

# print learning rate of benchmark model

print(benchMarkModel.optimizer.lr)

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

# Fine-tune the model using your smaller dataset
# Assume you have 'small_train_data', 'small_train_labels', 'small_val_data', and 'small_val_labels'
history = benchMarkModel.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=250,
                    batch_size=16,
                    callbacks=[early_stopping])

# Evaluate the model performance
val_loss = benchMarkModel.evaluate(X_test, y_test)
print(f'Validation loss: {val_loss}')

benchMarkAccuracy,results_benchmark = eval_model(X_test, y_test, benchMarkModel,first_cluster)


In [ ]:
import plotly.graph_objects as go
def visualize_future_distribution(results):
    '''
    Create stacked box and whisker plots for the predicted and real values
    '''

    fig = go.Figure()
    print(results.shape)

    for i in range(6):

        fig.add_trace(go.Box(y=results[f'predicted_{i+1}'], name=f'Predicted {i}')) 
        fig.add_trace(go.Box(y=results[f'real_{i+1}'], name=f'Real {i}'))

    fig.update_layout(
        title='Future Performance of Cluster',
        xaxis_title='Steps in future',
        yaxis_title='Cumulative Percent Change'
    ) 

    return fig

In [ ]:
from plotly.subplots import make_subplots
bench_fig = visualize_future_distribution(results_benchmark)
tuned_fig = visualize_future_distribution(results_tuned)
fig = make_subplots(rows=1, cols=2)

for trace in bench_fig.data:
    fig.add_trace(trace, row=1, col=1)

for trace in tuned_fig.data:
    fig.add_trace(trace, row=1, col=2)

fig.show()

In [ ]:
print(benchMarkAccuracy)
print(fineTunedAccuracy)